In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display


import numpy as np
import scipy as sp
import pandas as pd
import numpy.random as rand
import numpy.linalg as la
import scipy.optimize as opt
import sympy as sym

import matplotlib.pyplot as plt
import seaborn as sns


## One-way Push Model

In [ ]:
def one_way_push(x, args):
    
    (WT, ST) = x
    
    (vWS, vWSp, vSp) = args
    
    p = np.zeros(3, float)
    p[0] = vWS
    p[1] = 1 + vWS*(WT-ST)
    p[2] = -ST
    
    roots = np.roots(p)
    
    nonzero = np.nonzero(roots>=0.0)
    if len(nonzero) > 1:
        print("Multiple valid roots:", roots)
    
    Sf = roots[nonzero][0]
    
    W = WT/(1+vWS*Sf)
    
    pWSu = vWS*W/(1+vWS*W)
    
    SpT = ST*(vWSp*pWSu + vSp)/ (vWSp*pWSu + vSp + 1)
    SuT = ST - SpT
    
    Sp = SpT/(1+vWS*W)
    Su = SuT/(1+vWS*W)
    
    WSu = pWSu*SuT
    WSp = WT - W - WSu
        
    return np.array([SpT, SuT, W, Sp, Su, WSp, WSu])
    
    
    
    

In [ ]:
WT = 1
ST = 1
vWS = 10.0
# vWSp = 10.0
vSp = 1.0

N = 100

x = np.zeros([N, 7])

vWSp_list = np.logspace(-3, 3, N)
for i, vWSp in enumerate(vWSp_list):
    x[i] = one_way_push((WT, ST), (vWS, vWSp, vSp))


In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':True})


fig, ax = plt.subplots(1,1, figsize=(6, 4))

ax.plot(vWSp_list, (x[:, 0])/ST, label="$[S^p_T]/[S_T]$")
ax.plot(vWSp_list, (x[:, 1])/ST, label="$[S^u_T]/[S_T]$")
ax.plot(vWSp_list, (x[:, 2])/WT, label="$[W]/[W_T]$")
ax.plot(vWSp_list, (x[:, 3])/ST, label="$[S^p]/[S_T]$")
ax.plot(vWSp_list, (x[:, 4])/ST, label="$[S^u]/[S_T]$")

ax.plot(vWSp_list, (x[:, 5])/ST, label="$[WS^p]/[S_T]$")
ax.plot(vWSp_list, (x[:, 6])/ST, label="$[WS^u]/[S_T]$")






ax.set_xscale('log')

ax.set_ylabel("Fraction")
ax.set_xlabel("Phosphorylation Velocity $v_{WS}^p$")

ax.set_ylim(-0.1, 1.1)


ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt.show()

## Goldbeter Model

In [ ]:

def Goldbeter_push_pull(x, args):
    (WT, ET, ST, vWS, vES, vWSp) = args
    
    (W, E, Su, Sp, WSu, ESp, SuT, SpT) = x
        
    res = np.zeros_like(x)
    
    
    pWSu = vWS*W / (1 + vWS*W)
    pESp = vES*E / (1 + vES*E)
    
    res[0] = SpT/ST - vWSp*pWSu/ (vWSp*pWSu + pESp)
    res[1] = W/WT - 1/(1+vWS*Su)
    res[2] = E/ET - 1/(1+vES*Sp)
    res[3] = Su/SuT - 1/(1+vWS*W)
    res[4] = Sp/SpT - 1/(1+vES*E)
    res[5] = ST - SuT - SpT
    res[6] = SuT - Su - WSu
    res[7] = SpT - Sp - ESp
    
#     print(res)
            
    return res


In [ ]:

WT = 10
ET = 10
ST = 10
vWS = 1.0
vES = 1.0

W0 = 7.0
E0 = 7.0
Su0 = 2.0
Sp0 = 2.0
WSu0 = 3.0
ESp0 = 3.0
SuT0 = 5.0
SpT0 = 5.0

N = 100

x0 = (W0, E0, Su0, Sp0, WSu0, ESp0, SuT0, SpT0)

x = np.zeros([N, len(x0)])

vWSp_list = np.logspace(-3, 3, N)
for i, vWSp in enumerate(vWSp_list):
    
    args = ((WT, ET, ST, vWS, vES, vWSp),)
    
    result = opt.root(Goldbeter_push_pull, x0, args=args, method='lm')
    
    x[i] = result.x
    
#     print(result)
    
#     break
    


In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':True})


fig, ax = plt.subplots(1,1, figsize=(6, 4))

ax.plot(vWSp_list, x[:, 7]/ST, label="$[S^p_T]/[S_T]$")
ax.plot(vWSp_list, x[:, 6]/ST, label="$[S^u_T]/[S_T]$")
# ax.plot(vWSp_list, x[:, 5]/ST, label="$[ES^p]/[S_T]$")
# ax.plot(vWSp_list, x[:, 4]/ST, label="$[WS^u]/[S_T]$")
# ax.plot(vWSp_list, x[:, 3]/ST, label="$[S^p]/[S_T]$")
# ax.plot(vWSp_list, x[:, 2]/ST, label="$[S^u]/[S_T]$")
# ax.plot(vWSp_list, (x[:, 1])/ET, label="$[E]/[E_T]$")
# ax.plot(vWSp_list, (x[:, 0])/WT, label="$[W]/[W_T]$")


ax.set_xscale('log')

ax.set_ylabel("Fraction")
ax.set_xlabel("Phosphorylation Velocity $v_{WS}^p$")


ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt.show()

## Full Push-Pull Model

In [ ]:
def full_push_pull(x, args):
    
    (WT, ET, ST) = x
    
    (vWS, vES, vWSp, vESu, vSp) = args
    
#     (W, E, Sp, Su, WSp, WSu, ESp, ESu, SpT, SuT) = x

    p = np.zeros(4, float)
    p[0] = vWS*vES
    p[1] = vWS + vES + vWS*vES*(WT+ET-ST)
    p[2] = 1 + vWS*WT + vES*ET - (vWS+vES)*ST
    p[3] = -ST
    
    roots = np.roots(p)
    
    nonzero = np.nonzero(roots>=0.0)
    if len(nonzero) > 1:
        print("Multiple valid roots:", roots)
    
    Sf = roots[nonzero][0]
    
    W = WT/(1+vWS*Sf)
    E =  ET/(1+vES*Sf)    
    
    pWSu = vWS*W/(1+vWS*W+vES*E)
    pESp = vES*E/(1+vWS*W+vES*E)
    
    SpT = ST*(vWSp*pWSu + vSp)/ (vWSp*pWSu + vESu*pESp + vSp + 1)
    SuT = ST - SpT
    
    Sp = SpT/(1+vWS*W+vES*E)
    Su = SuT/(1+vWS*W+vES*E)
    
    WSu = pWSu*SuT
    WSp = WT - W - WSu
    
    ESp = pESp*SpT
    ESu = ET -E - ESp
        
    return np.array([SpT, SuT, W, E, Sp, Su, WSp, WSu, ESp, ESu])
    
    
    
    

In [ ]:
WT = 1
# ET = 10
ST = 1
vWS = 10.0
vES = 10.0
vESu = 10.0
vWSp = 10.0
vSp = 1.0

N = 100

x = np.zeros([N, 10])

ET_list = np.logspace(-3, 3, N)
# ET_list = [1e1, 10**1.5, 1e2, 1e3]
# ET_list = [1e1]
for i, ET in enumerate(ET_list):
    x[i] = full_push_pull((WT, ET, ST), (vWS, vES, vWSp, vESu, vSp))


In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':True})


fig, ax = plt.subplots(1,1, figsize=(6, 4))

ax.plot(ET_list, (x[:, 0])/ST, label="$[S^p_T]/[S_T]$")
ax.plot(ET_list, (x[:, 1])/ST, label="$[S^u_T]/[S_T]$")
ax.plot(ET_list, (x[:, 2])/WT, label="$[W]/[W_T]$")
ax.plot(ET_list, (x[:, 3])/np.array(ET_list), label="$[E]/[E_T]$")
ax.plot(ET_list, (x[:, 4])/ST, label="$[S^p]/[S_T]$")
ax.plot(ET_list, (x[:, 5])/ST, label="$[S^u]/[S_T]$")

# ax.plot(ET_list, (x[:, 6])/ST, label="$[WS^p]/[S_T]$")
# ax.plot(ET_list, (x[:, 7])/ST, label="$[WS^u]/[S_T]$")
# ax.plot(ET_list, (x[:, 8])/ST, label="$[ES^p]/[S_T]$")
# ax.plot(ET_list, (x[:, 9])/ST, label="$[ES^u]/[S_T]$")






ax.set_xscale('log')

ax.set_ylabel("Fraction")
ax.set_xlabel("Total Eraser Concentration $[E_T]$")

ax.set_ylim(-0.1, 1.1)


ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt.show()

In [ ]:
WT = 1
ET = 1
ST = 1
vWS = 10.0
vES = 10.0
vESu = 10.0
# vWSp = 10.0
vSp = 1.0

N = 100

x = np.zeros([N, 10])

vWSp_list = np.logspace(-3, 3, N)
for i, vWSp in enumerate(vWSp_list):
    x[i] = full_push_pull((WT, ET, ST), (vWS, vES, vWSp, vESu, vSp))


In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':True})


fig, ax = plt.subplots(1,1, figsize=(6, 4))

ax.plot(vWSp_list, (x[:, 0])/ST, label="$[S^p_T]/[S_T]$")
ax.plot(vWSp_list, (x[:, 1])/ST, label="$[S^u_T]/[S_T]$")
ax.plot(vWSp_list, (x[:, 2])/WT, label="$[W]/[W_T]$")
ax.plot(vWSp_list, (x[:, 3])/ET, label="$[E]/[E_T]$")
ax.plot(vWSp_list, (x[:, 4])/ST, label="$[S^p]/[S_T]$")
ax.plot(vWSp_list, (x[:, 5])/ST, label="$[S^u]/[S_T]$")

# ax.plot(vWSp_list, (x[:, 6])/ST, label="$[WS^p]/[S_T]$")
# ax.plot(vWSp_list, (x[:, 7])/ST, label="$[WS^u]/[S_T]$")
# ax.plot(vWSp_list, (x[:, 8])/ST, label="$[ES^p]/[S_T]$")
# ax.plot(vWSp_list, (x[:, 9])/ST, label="$[ES^u]/[S_T]$")






ax.set_xscale('log')

ax.set_ylabel("Fraction")
ax.set_xlabel("Phosphorylation Velocity $v_{WS}^p$")

ax.set_ylim(-0.1, 1.1)


ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt.show()